In [9]:
from transformers import pipeline, Conversation, AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "facebook/blenderbot-1B-distill"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


if torch.cuda.is_available():
    print("cuda available, using cuda!")
    model = model.to("cuda")

#generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # takes upwards of 1 min, immeseured.
#generator = pipeline("conversational", model=model_name)     # 12 sec
generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # 15sec

In [10]:
convo = Conversation("Hi, what's your name?")


In [11]:
response = generator(convo)   #takes 12 seconds with just model=model_name as param

In [12]:
response

Conversation id: f5d3b8b6-b7a0-4734-8f5a-a8d6931c61c1
user: Hi, what's your name?
assistant:  I I Do I   I, of course, like me?

In [13]:
convo.add_user_input("Alrighty")
convo = generator(convo)

In [14]:
convo

Conversation id: f5d3b8b6-b7a0-4734-8f5a-a8d6931c61c1
user: Hi, what's your name?
assistant:  I I Do I   I, of course, like me?
user: Alrighty
assistant: 

Now to add a way for user inputs

In [15]:
import gradio as gr

In [16]:
inputs = []
responses = []
# inputs and responses could be combined to form a 'memory' of sorts
# how to implement backstory/key memories? need to research

def convo_bot(input, response):
    convo = Conversation(text=input, past_user_inputs=inputs, generated_responses=responses)
    convo = generator(convo)
    return convo.generated_responses[-1]

demo = gr.ChatInterface(convo_bot, title="Vanilla Chatbot", description="...")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
